**Creating environment and basic training**

In [1]:
#Importing libraries
import gymnasium as gym
from stable_baselines3 import PPO
import os

# Setting up Environment
env_id = "BipedalWalker-v3"

def train():
    env = gym.make(env_id)

    # Initializing the Agent, using PPO with a standard MLP policy.
    model = PPO("MlpPolicy", env, verbose=1, tensorboard_log="./ppo_bipedal_logs/")

    # Training the Agent
    # For a bipedal walker to walk, we need ~500,000 to 1M steps.
    print("Starting training...")
    model.learn(total_timesteps=700000)

    # Saving the model
    model.save("ppo_bipedal_walker")
    print("Model saved.")
    env.close()

def render_ai():
    #Load and Render
    print("Loading model for rendering...")
    env = gym.make(env_id, render_mode="human")
    model = PPO.load("ppo_bipedal_walker")

    obs, info = env.reset()
    for _ in range(1000):
        # Predict the action based on the observation, deterministic=True for better evaluation
        action, _states = model.predict(obs, deterministic=True)

        # Apply the action to the environment
        obs, reward, terminated, truncated, info = env.step(action)

        env.render()

        if terminated or truncated:
            obs, info = env.reset()

    env.close()

if __name__ == "__main__":
    train()
    render_ai()

/Users/ShivenTiwari/Desktop/Bipedal_Walker/venv/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Starting training...
Logging to ./ppo_bipedal_logs/PPO_6
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 836      |
|    ep_rew_mean     | -113     |
| time/              |          |
|    fps             | 3959     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 510         |
|    ep_rew_mean          | -107        |
| time/                   |             |
|    fps                  | 2579        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011635919 |
|    clip_fraction        | 0.119       |
|    

In [ ]:
#Recording result after setting up environment and initial training
from record import record
record()

/Users/ShivenTiwari/Desktop/Bipedal_Walker/venv/lib/python3.13/site-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /Users/ShivenTiwari/Desktop/Bipedal_Walker/recordings folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Video saved to: recordings


In [ ]:
#Score after baseline training

def evaluate_model():
    import gymnasium as gym
    from stable_baselines3 import PPO
    from stable_baselines3.common.evaluation import evaluate_policy
    from stable_baselines3.common.monitor import Monitor

    model = PPO.load("ppo_bipedal_walker")  # no .zip

    eval_env = gym.make("BipedalWalker-v3", render_mode="rgb_array")
    eval_env = Monitor(eval_env)

    mean_reward, std_reward = evaluate_policy(
    model,
    eval_env,
    n_eval_episodes=25,
    deterministic=True
    )

    print(f"Current score: {mean_reward:.2f} ± {std_reward:.2f}")
    eval_env.close()

evaluate_model()

#We see a great result of around 285...this indicates that our model is able to walk appropriately. The model is almost already perfect


Current score: 284.97 ± 56.65


**REWARD CRAFTING**

In [6]:
#Importing libraries
import gymnasium as gym
from stable_baselines3 import PPO
import os

class WalkerShaping(gym.Wrapper):
    def __init__(self, env, angle_k=0.2, angvel_k=0.05, alive_bonus=0.01, fall_penalty=5.0):
        super().__init__(env)
        self.angle_k = angle_k
        self.angvel_k = angvel_k
        self.alive_bonus = alive_bonus
        self.fall_penalty = fall_penalty

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)

        hull_angle = float(obs[0])
        hull_angvel = float(obs[1])

        shaped = reward
        shaped += self.alive_bonus
        shaped -= self.angle_k * abs(hull_angle)
        shaped -= self.angvel_k * abs(hull_angvel)

        if terminated:
            shaped -= self.fall_penalty

        return obs, shaped, terminated, truncated, info

env = gym.make("BipedalWalker-v3", render_mode="rgb_array")
env = WalkerShaping(env)
model = PPO("MlpPolicy", env, verbose=1)
model.learn(500000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 583      |
|    ep_rew_mean     | -137     |
| time/              |          |
|    fps             | 4554     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 684         |
|    ep_rew_mean          | -143        |
| time/                   |             |
|    fps                  | 3179        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008501378 |
|    clip_fraction        | 0.0933      |
|    clip_range           | 0.2         |
|    entropy_loss   

In [ ]:
#Record resulting after reward crafting
from record import record
record()

Video saved to: recordings


In [20]:
#Finding score
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

model = PPO.load("ppo_bipedal_walker")  # no .zip

eval_env = gym.make("BipedalWalker-v3", render_mode="rgb_array")
eval_env = Monitor(eval_env)

mean_reward, std_reward = evaluate_policy(
    model,
    eval_env,
    n_eval_episodes=25,
    deterministic=True
    )

print(f"Current score: {mean_reward:.2f} ± {std_reward:.2f}")
eval_env.close()
#Score increased by around 6 points but this might not indicate a strict increase since the new score is within the standard deviation of the previous score

Current score: 290.55 ± 49.53


Experiment 1 (Exploration v. Exploitation)

**GOAL** 
Test how changing PPO's entropy coefficient (ent_coef) affects learning.

**Concepts Involved:** 
ent_coef adds "entropy bonus" to PPO's objective.
Lower entropy encourages the policy to become deterministic sooner --> more exploitation.
Higher entropy encourages more stochastic actions --> more exploration.

**Hypothesis**
ent_coef = 0.00: The AI has no curiosity. As soon as it finds a potential solution, it follows it instantly. Gets stuck in a local optimum.
ent_coef = 0.01: The AI is compelled to stay random. Makes early learning messy and slow, because it tries multiple variations. More likely to find global optimum.

**Training Budget**
200,000 steps each

In policy-gradient methods, entropy regularization is commonly used to encourage exploration by penalizing overly confident (low-entropy) action distributions. Higher entropy promotes broader exploration, reducing the risk of premature convergence to suboptimal policies, while lower entropy allows the policy to refine and exploit learned behaviors. Prior work on PPO shows that carefully tuning the entropy coefficient is crucial for balancing exploration and stability, particularly in continuous-control tasks such as  movement. In this experiment, we vary the entropy coefficient to study its effect on learning dynamics and final walking performance in the BipedalWalker environment. (Schulman et al., 2017)

In [ ]:

from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor

# What we save:
# TensorBoard logs: training curves (reward proxy, losses, entropy loss)
# EvalCallback logs: evaluation mean reward over time
# Best model checkpoint per experiment

# Output directories
os.makedirs("logs/entropy_exp", exist_ok=True)
os.makedirs("logs/eval_stats", exist_ok=True)
os.makedirs("models", exist_ok=True)

ENV_ID = "BipedalWalker-v3"


def make_env(seed: int):
    """
    Creates a single environment instance wrapped with Monitor.
    Monitor is recommended by SB3 so evaluation metrics are recorded cleanly.
    """
    env = gym.make(ENV_ID)
    env = Monitor(env)
    env.reset(seed=seed)
    return env

def run_experiment(exp_name: str, entropy_value: float, total_timesteps: int = 200_000, seed: int = 0):
    """
    Runs PPO training for one entropy coefficient value.

    Args:
        exp_name: Name for logging and saving artifacts.
        entropy_value: PPO ent_coef (independent variable).
        total_timesteps: Total training steps.
        seed: Controls randomness for reproducibility.
    """
    print(f"\n--- Starting Experiment: {exp_name} (ent_coef={entropy_value}, seed={seed}) ---")

    # Create separate train and eval environments with different seeds so evaluation is not the same randomness as training
    train_env = make_env(seed=seed)
    eval_env = make_env(seed=seed + 10_000)

    # Define PPO model (MLP policy)
    # ent_coef is the only change between experiments (clean concept test)
    model = PPO(
        "MlpPolicy",
        train_env,
        verbose=0,
        seed=seed,
        ent_coef=entropy_value,
        tensorboard_log="./logs/entropy_exp",
    )

    # Evaluation callback:
    # evaluates every eval_freq steps
    # saves best model checkpoint based on mean eval reward
    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path=f"./models/{exp_name}",
        log_path=f"./logs/eval_stats/{exp_name}",
        eval_freq=10_000,          # evaluation interval
        n_eval_episodes=5,         # reduce runtime; increase to 10 for final results
        deterministic=True,
        render=False
    )

    # Training
    model.learn(
        total_timesteps=total_timesteps,
        callback=eval_callback,
        tb_log_name=exp_name
    )

    # Save final model (not just best checkpoint)
    model.save(f"./models/{exp_name}_final")

    # Cleanup
    train_env.close()
    eval_env.close()

    print(f"--- Finished: {exp_name} ---")


if __name__ == "__main__":
    # Screening: 200k steps, 1 seed, 5 eval episodes
    # Final: 500k steps, 2–3 seeds, 10 eval episodes

    run_experiment("Baseline_NoEntropy", entropy_value=0.0, total_timesteps=200_000, seed=0)
    run_experiment("Exploration_0.01", entropy_value=0.01, total_timesteps=200_000, seed=0)

    print("\nDONE!")




--- Starting Experiment: Baseline_NoEntropy (ent_coef=0.0, seed=0) ---
Eval num_timesteps=10000, episode_reward=-72.76 +/- 26.45
Episode length: 706.60 +/- 729.52
New best mean reward!
Eval num_timesteps=20000, episode_reward=-82.08 +/- 17.84
Episode length: 1308.40 +/- 583.20
Eval num_timesteps=30000, episode_reward=-95.75 +/- 0.28
Episode length: 111.80 +/- 1.72
Eval num_timesteps=40000, episode_reward=-69.13 +/- 0.04
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=50000, episode_reward=-45.64 +/- 0.94
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=60000, episode_reward=-33.14 +/- 0.84
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=70000, episode_reward=-35.63 +/- 2.93
Episode length: 1600.00 +/- 0.00
Eval num_timesteps=80000, episode_reward=30.82 +/- 43.83
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=90000, episode_reward=107.19 +/- 3.18
Episode length: 1600.00 +/- 0.00
New 

RESULT (for experiment 1): I evaluated the effect of different entropy on PPO performance in BipedalWalker by comparing entropy coef = 0.0 and 0.01. While increased entropy encouraged exploration, it significantly reduced intial stability and delayed learning. Under a limited training budget (100–200k timesteps), the baseline configuration consistently achieved higher evaluation rewards and longer episodes. As a result, retaining the entropy coef of 0.0 for subsequent experiments is more effective for our training budget.

These aspects are visible in the graph, csv file, and GIFs. There is significant differance in visible behavior and data points.

In [ ]:
#CSV to store data points for entropy 0.00 and 0.01 (exploitation and exploration respectively)
import numpy as np
import pandas as pd
EVAL_DIR = "./logs/eval_stats"
OUT_CSV = "entropy_experiment_results.csv"

experiments = [
    {"name": "Baseline_NoEntropy", "ent_coef": 0.0},
    {"name": "Exploration_0.01", "ent_coef": 0.01},
]

rows = []

for exp in experiments:
    exp_name = exp["name"]
    npz_path = os.path.join(EVAL_DIR, exp_name, "evaluations.npz")

    if not os.path.exists(npz_path):
        print(f"Missing: {npz_path}")
        continue

    data = np.load(npz_path)

    # SB3 EvalCallback saves arrays like:
    # timesteps: shape (K,)
    # results:   shape (K, n_eval_episodes)
    timesteps = data["timesteps"]
    results = data["results"]

    mean_rewards = results.mean(axis=1)
    std_rewards = results.std(axis=1)

    best_idx = int(np.argmax(mean_rewards))

    rows.append({
        "Experiment": exp_name,
        "ent_coef": exp["ent_coef"],
        "Final_Timestep": int(timesteps[-1]),
        "Final_Eval_MeanReward": float(mean_rewards[-1]),
        "Final_Eval_StdReward": float(std_rewards[-1]),
        "Best_Timestep": int(timesteps[best_idx]),
        "Best_Eval_MeanReward": float(mean_rewards[best_idx]),
        "Best_Eval_StdReward": float(std_rewards[best_idx]),
        "Best_Model_Path": f"./models/{exp_name}/best_model.zip",
    })

df = pd.DataFrame(rows)
df.to_csv(OUT_CSV, index=False)
print(df)
print(f"\nSaved CSV to: {OUT_CSV}")


           Experiment  ent_coef  Final_Timestep  Final_Eval_MeanReward  \
0  Baseline_NoEntropy      0.00          200000             202.571684   
1    Exploration_0.01      0.01          200000             192.365161   

   Final_Eval_StdReward  Best_Timestep  Best_Eval_MeanReward  \
0            129.629802         180000            211.344107   
1             69.338017         200000            192.365161   

   Best_Eval_StdReward                             Best_Model_Path  
0           104.995475  ./models/Baseline_NoEntropy/best_model.zip  
1            69.338017    ./models/Exploration_0.01/best_model.zip  

Saved CSV to: entropy_experiment_results.csv


In [ ]:
#GIFs to understand behavior of both bipedal walkers

import imageio

ENV_ID = "BipedalWalker-v3"
GIF_DIR = "./gifs"
os.makedirs(GIF_DIR, exist_ok=True)

experiments = [
    "Baseline_NoEntropy",
    "Exploration_0.01",
]

def render_model_to_gif(model_path: str, out_gif: str, seed: int = 123, max_steps: int = 1600):
    # Use rgb_array to capture frames
    env = gym.make(ENV_ID, render_mode="rgb_array")
    obs, _ = env.reset(seed=seed)

    model = PPO.load(model_path)

    frames = []
    for _ in range(max_steps):
        frame = env.render()
        if frame is not None:
            frames.append(frame)

        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)

        if terminated or truncated:
            # capture one last frame
            frame = env.render()
            if frame is not None:
                frames.append(frame)
            break

    env.close()

    # Save gif (fps can be adjusted)
    imageio.mimsave(out_gif, frames, fps=30)
    print(f"Saved: {out_gif}  ({len(frames)} frames)")

if __name__ == "__main__":
    for exp_name in experiments:
        model_path = f"./models/{exp_name}/best_model.zip"
        out_gif = os.path.join(GIF_DIR, f"{exp_name}.gif")

        if not os.path.exists(model_path):
            print(f"Missing model: {model_path}")
            continue

        render_model_to_gif(model_path, out_gif)


Saved: ./gifs/Baseline_NoEntropy.gif  (1601 frames)
Saved: ./gifs/Exploration_0.01.gif  (1601 frames)


**Experiment 2 (Observation normalization/ VecNormalize)**

**GOAL** 
Test how VecNormalization affects stability and learning

**Concepts Involved:** 
Observations such as angles, velocities etc. have different scales
VecNormalize standardizes these observations using a "running mean", calculation of mean on subsets of observation metrics
This often stabilizes learning and sample effiency.

**Variables**
A) No normalization (baseline)
B) VecNormalize(norm_obs=True, norm_reward=False)

**Training budget**
200,000 steps

Reinforcement learning algorithms are sensitive to the scale of input features, as large differences in observation magnitudes can lead to unstable gradients and slow or inconsistent learning. Observation normalization addresses this issue by standardizing state inputs, reducing internal covariate shift and improving optimization stability. Prior work in deep learning and policy-gradient methods shows that normalization techniques improve convergence and robustness, particularly in continuous-control tasks(Ioffe & Szegedy, 2015; Schulman et al., 2017)

In [1]:
import os
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback, BaseCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

os.makedirs("logs/obsnorm_exp", exist_ok=True)
os.makedirs("logs/eval_stats", exist_ok=True)
os.makedirs("models", exist_ok=True)

ENV_ID = "BipedalWalker-v3"
TOTAL_TIMESTEPS = 200000
EVAL_FREQ = 50_000
N_EVAL_EPISODES = 3

class SyncObsStatsCallback(BaseCallback):
    """
    Copies running observation stats (obs_rms) from train_env -> eval_env.
    Needed when using VecNormalize so eval uses the same normalization.
    """
    def __init__(self, train_env, eval_env, sync_every: int = 1000):
        super().__init__()
        self.train_env = train_env
        self.eval_env = eval_env
        self.sync_every = sync_every

    def _on_step(self) -> bool:
        # Sync periodically to avoid pointless per-step overhead
        if self.n_calls % self.sync_every == 0:
            self.eval_env.obs_rms = self.train_env.obs_rms
        return True


def make_raw_env(seed: int):
    env = gym.make(ENV_ID)
    env = Monitor(env)
    env.reset(seed=seed)
    return env


def make_vec_env(seed: int):
    return DummyVecEnv([lambda: make_raw_env(seed)])


def run_experiment(exp_name: str, use_obs_norm: bool, seed: int = 0):
    print(f"\n--- Starting Experiment: {exp_name} (ObsNorm={use_obs_norm}, seed={seed}) ---")

    train_env = make_vec_env(seed=seed)
    eval_env = make_vec_env(seed=seed + 10_000)

    current_callbacks = []

    if use_obs_norm:
        train_env = VecNormalize(train_env, norm_obs=True, norm_reward=False, clip_obs=10.0)
        eval_env = VecNormalize(eval_env, norm_obs=True, norm_reward=False, clip_obs=10.0)
        eval_env.training = False

        current_callbacks.append(SyncObsStatsCallback(train_env, eval_env, sync_every=1000))

    model = PPO(
        "MlpPolicy",
        train_env,
        verbose=0,
        seed=seed,
        learning_rate=3e-4,
        n_steps=2048,
        batch_size=64,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        ent_coef=0.0,
        tensorboard_log="./logs/obsnorm_exp",
    )

    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path=f"./models/{exp_name}",
        log_path=f"./logs/eval_stats/{exp_name}",
        eval_freq=EVAL_FREQ,
        n_eval_episodes=N_EVAL_EPISODES,
        deterministic=True,
        render=False,
    )

    current_callbacks.append(eval_callback)

    model.learn(
        total_timesteps=TOTAL_TIMESTEPS,
        callback=current_callbacks,
        tb_log_name=exp_name,
    )

    if use_obs_norm:
        train_env.save(f"./models/{exp_name}/vecnormalize.pkl")

    model.save(f"./models/{exp_name}_final")

    train_env.close()
    eval_env.close()
    print(f"--- Finished: {exp_name} ---")


if __name__ == "__main__":
    run_experiment("NoObsNorm", use_obs_norm=False, seed=0)
    run_experiment("ObsNorm_On", use_obs_norm=True, seed=0)
    print("\nWe are done.")




--- Starting Experiment: NoObsNorm (ObsNorm=False, seed=0) ---


/Users/ShivenTiwari/Desktop/Bipedal_Walker/venv/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Eval num_timesteps=50000, episode_reward=-46.88 +/- 0.50
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=100000, episode_reward=116.38 +/- 0.81
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=150000, episode_reward=-62.51 +/- 10.46
Episode length: 267.67 +/- 52.36
Eval num_timesteps=200000, episode_reward=225.25 +/- 58.33
Episode length: 1524.67 +/- 55.24
New best mean reward!
--- Finished: NoObsNorm ---

--- Starting Experiment: ObsNorm_On (ObsNorm=True, seed=0) ---
Eval num_timesteps=50000, episode_reward=-94.60 +/- 11.26
Episode length: 270.33 +/- 107.21
New best mean reward!
Eval num_timesteps=100000, episode_reward=-12.17 +/- 28.18
Episode length: 554.00 +/- 178.05
New best mean reward!
Eval num_timesteps=150000, episode_reward=262.42 +/- 6.81
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=200000, episode_reward=67.40 +/- 140.88
Episode length: 856.33 +/- 526.05
--- Finished: ObsNorm_On ---

We are done

RESULT (for experiment 2): Observation normalization substantially improves PPO performance on BipedalWalker by enhancing stability and forward movement efficiency.
Although normalized agents exhibit shorter episode lengths, they achieve significantly higher rewards under identical training conditions, demonstrating more effective use of each timestep. 

In [ ]:
#Getting GIFS for no normalization and normalization

import imageio
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

ENV_ID = "BipedalWalker-v3"
GIF_DIR = "./gifs"
os.makedirs(GIF_DIR, exist_ok=True)

EXPERIMENTS = [
    ("NoObsNorm", False),
    ("ObsNorm_On", True),
]

def render_model_to_gif(model_dir: str, out_gif: str, use_obs_norm: bool,
                        seed: int = 123, max_steps: int = 1600, fps: int = 30):

    def make_env():
        return gym.make(ENV_ID, render_mode="rgb_array")

    # Use VecEnv wrapper for SB3 consistency
    env = DummyVecEnv([make_env])

    # Load normalization stats if model was trained with VecNormalize
    if use_obs_norm:
        vecnorm_path = os.path.join(model_dir, "vecnormalize.pkl")
        if not os.path.exists(vecnorm_path):
            raise FileNotFoundError(f"Missing VecNormalize stats: {vecnorm_path}")

        env = VecNormalize.load(vecnorm_path, env)
        env.training = False
        env.norm_reward = False

    model_path = os.path.join(model_dir, "best_model.zip")
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Missing model: {model_path}")

    model = PPO.load(model_path)

    obs = env.reset()
    frames = []

    for _ in range(max_steps):
        frame = env.render()
        if frame is not None:
            frames.append(frame)

        action, _ = model.predict(obs, deterministic=True)
        obs, _, done, _ = env.step(action)

        if done:
            frame = env.render()
            if frame is not None:
                frames.append(frame)
            break

    env.close()

    imageio.mimsave(out_gif, frames, fps=fps)
    print(f"Saved: {out_gif} ({len(frames)} frames)")

if __name__ == "__main__":
    for exp_name, use_obs_norm in EXPERIMENTS:
        model_dir = f"./models/{exp_name}"
        out_gif = os.path.join(GIF_DIR, f"{exp_name}.gif")

        if not os.path.isdir(model_dir):
            print(f"Missing directory: {model_dir}")
            continue

        render_model_to_gif(model_dir, out_gif, use_obs_norm)


Saved: ./gifs/NoObsNorm.gif (1465 frames)
Saved: ./gifs/ObsNorm_On.gif (1429 frames)


In [4]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy
import os

ENV_ID = "BipedalWalker-v3"

def eval_model(exp_name, use_obs_norm):
    def make_env():
        return gym.make(ENV_ID)
    env = DummyVecEnv([make_env])

    model_dir = f"./models/{exp_name}"
    model = PPO.load(os.path.join(model_dir, "best_model.zip"))

    if use_obs_norm:
        env = VecNormalize.load(os.path.join(model_dir, "vecnormalize.pkl"), env)
        env.training = False
        env.norm_reward = False

    mean, std = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)
    env.close()
    print(exp_name, "mean=", mean, "std=", std)

eval_model("NoObsNorm", False)
eval_model("ObsNorm_On", True)



/Users/ShivenTiwari/Desktop/Bipedal_Walker/venv/lib/python3.13/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


NoObsNorm mean= 228.9270911990199 std= 70.35932071500235
ObsNorm_On mean= 265.9788552279002 std= 4.127669522396412


**Experiment 4 (PPO Hyperparameter Sensitivity -- Learning Rate & Batch Size)**

**GOAL**
Identify PPO hyperparameters that lead to stable and scalable learning for BipedalWalker before training the final model.

**Concepts involved**
PPO updates can be too aggressive or too conservative, affecting balance and stability.
Learning rate controls how large each policy update is.
Batch size controls how noisy or smooth gradient updates are.
Observation normalization (VecNormalize) is kept on to ensure differences come only from PPO update behavior.

**Variables**
Learning rate: 1e-4, 3e-4, 1e-3
Batch size: 64, 128

**Fixed settings**
PPO with MLP policy
Entropy coefficient = 0.0
n_steps = 2048
4 parallel environments
Observation normalization enabled

**Training budget**
300,000 steps per configuration

In [ ]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback

ENV_ID = "BipedalWalker-v3"
LOG_DIR = "./bipedal_tuning_logs"
MODEL_DIR = "./models_tuning"
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

# Fixed constants
ENT_COEF = 0.0
TOTAL_TIMESTEPS = 300_000
EVAL_FREQ = 10_000

EXPERIMENTS = [
    {"name": "1_Baseline_LR3e-4_B64",   "lr": 3e-4, "batch": 64},
    {"name": "2_Cautious_LR1e-4_B64",   "lr": 1e-4, "batch": 64},
    {"name": "3_Aggressive_LR1e-3_B64", "lr": 1e-3, "batch": 64},
    {"name": "4_BigBatch_LR3e-4_B128",  "lr": 3e-4, "batch": 128},
    {"name": "5_Precise_LR1e-4_B128",   "lr": 1e-4, "batch": 128},
]

def make_env():
    return gym.make(ENV_ID)

if __name__ == "__main__":
    for exp in EXPERIMENTS:
        exp_name = exp["name"]
        lr = exp["lr"]
        batch_size = exp["batch"]

        print(f"--- Running: {exp_name} (LR={lr}, Batch={batch_size}) ---")

        save_path = os.path.join(MODEL_DIR, exp_name)
        os.makedirs(save_path, exist_ok=True)

        # Training environment
        train_env = DummyVecEnv([make_env for _ in range(4)])
        train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10.0)

        # Evaluation env
        #    - separate env
        #    - true reward (norm_reward=False)
        #    - share obs stats
        eval_env = DummyVecEnv([make_env])
        eval_env = VecNormalize(eval_env, norm_obs=True, norm_reward=False, clip_obs=10.0)

        # Share observation normalization stats so eval uses same scaling
        eval_env.obs_rms = train_env.obs_rms

        eval_callback = EvalCallback(
            eval_env,
            best_model_save_path=save_path,
            log_path=save_path,
            eval_freq=EVAL_FREQ,
            deterministic=True,
            n_eval_episodes=5
        )

        # Model
        model = PPO(
            "MlpPolicy",
            train_env,
            verbose=0,
            ent_coef=ENT_COEF,
            learning_rate=lr,
            batch_size=batch_size,
            n_steps=2048,
            tensorboard_log=LOG_DIR
        )

        # Train
        try:
            model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=eval_callback, tb_log_name=exp_name)
        except KeyboardInterrupt:
            print(f"Interrupted: {exp_name}")

        # 5) Saving!
        model.save(os.path.join(save_path, "final_model"))
        train_env.save(os.path.join(save_path, "vecnormalize.pkl"))  # CRITICAL

        train_env.close()
        eval_env.close()
        print(f"Completed: {exp_name}")

    print("All tuning experiments done.")

--- Running: 1_Baseline_LR3e-4_B64 (LR=0.0003, Batch=64) ---
Eval num_timesteps=40000, episode_reward=-17.86 +/- 1.28
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=80000, episode_reward=-50.86 +/- 3.36
Episode length: 1600.00 +/- 0.00
Eval num_timesteps=120000, episode_reward=49.28 +/- 13.61
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=160000, episode_reward=99.64 +/- 117.39
Episode length: 1300.20 +/- 568.14
New best mean reward!
Eval num_timesteps=200000, episode_reward=149.84 +/- 116.26
Episode length: 1357.80 +/- 419.72
New best mean reward!
Eval num_timesteps=240000, episode_reward=251.39 +/- 7.16
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=280000, episode_reward=247.93 +/- 12.47
Episode length: 1600.00 +/- 0.00
Completed: 1_Baseline_LR3e-4_B64
--- Running: 2_Cautious_LR1e-4_B64 (LR=0.0001, Batch=64) ---
Eval num_timesteps=40000, episode_reward=-15.54 +/- 5.55
Episode length: 1600.00 +/- 0.00
Ne

In [7]:
#Looking for best model
import os
import numpy as np

MODEL_DIR = "./models_tuning"

def summarize_run(exp_dir: str):
    p = os.path.join(MODEL_DIR, exp_dir, "evaluations.npz")
    if not os.path.exists(p):
        return None

    data = np.load(p)
    # results shape: (n_evals, n_eval_episodes)
    means = data["results"].mean(axis=1)
    stds  = data["results"].std(axis=1)
    steps = data["timesteps"]

    best_idx = int(np.argmax(means))
    best_mean = float(means[best_idx])
    best_step = int(steps[best_idx])

    # Stability: how much it drops after reaching best
    post_best = means[best_idx:]
    worst_after_best = float(post_best.min())
    drop = float(best_mean - worst_after_best)

    # End performance: average of last 3 eval points (more “final” than a single spike)
    k = min(3, len(means))
    end_avg = float(means[-k:].mean())

    # Trend: last eval minus first eval (rough improvement)
    trend = float(means[-1] - means[0])

    return {
        "name": exp_dir,
        "best_mean": best_mean,
        "best_step": best_step,
        "end_avg": end_avg,
        "drop_after_best": drop,
        "trend": trend,
        "n_evals": len(means),
    }

rows = []
for exp in sorted(os.listdir(MODEL_DIR)):
    if os.path.isdir(os.path.join(MODEL_DIR, exp)):
        s = summarize_run(exp)
        if s:
            rows.append(s)

if not rows:
    print("No evaluations.npz found. Check that EvalCallback ran and log_path is correct.")
    raise SystemExit

# Ranking score: prefer high end_avg + best_mean, penalize big collapses (drop)
# You can tweak weights, but this is a solid default.
for r in rows:
    r["score"] = (0.6 * r["end_avg"]) + (0.4 * r["best_mean"]) - (0.8 * r["drop_after_best"])

rows_sorted = sorted(rows, key=lambda x: x["score"], reverse=True)

print("=== Ranked (best first) ===")
for r in rows_sorted:
    print(
        f"{r['name']:<28} "
        f"score={r['score']:.1f}  "
        f"end_avg={r['end_avg']:.1f}  "
        f"best={r['best_mean']:.1f}@{r['best_step']}  "
        f"drop={r['drop_after_best']:.1f}  "
        f"trend={r['trend']:.1f}"
    )

print("\nWinner suggestion:", rows_sorted[0]["name"])


=== Ranked (best first) ===
1_Baseline_LR3e-4_B64        score=227.6  end_avg=216.4  best=251.4@240000  drop=3.5  trend=265.8
4_BigBatch_LR3e-4_B128       score=220.2  end_avg=194.4  best=258.9@280000  drop=0.0  trend=309.1
5_Precise_LR1e-4_B128        score=114.2  end_avg=112.0  best=117.6@280000  drop=0.0  trend=127.4
3_Aggressive_LR1e-3_B64      score=27.5  end_avg=47.5  best=128.4@240000  drop=65.5  trend=102.2
2_Cautious_LR1e-4_B64        score=-112.0  end_avg=30.7  best=166.9@240000  drop=246.5  trend=-64.0

Winner suggestion: 1_Baseline_LR3e-4_B64


In [8]:
#Let's now further train this model to see the score 

import gymnasium as gym
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback

ENV_ID = "BipedalWalker-v3"
EXP_PATH = "./models_tuning/1_Baseline_LR3e-4_B64"
TOTAL_ADDITIONAL_STEPS = 1_000_000   # can raise to 1.5M or 2M

def make_env():
    return gym.make(ENV_ID)

# Train env
train_env = DummyVecEnv([make_env for _ in range(4)])
train_env = VecNormalize.load(f"{EXP_PATH}/vecnormalize.pkl", train_env)
train_env.training = True
train_env.norm_reward = True

# Eval env
eval_env = DummyVecEnv([make_env])
eval_env = VecNormalize.load(f"{EXP_PATH}/vecnormalize.pkl", eval_env)
eval_env.training = False
eval_env.norm_reward = False

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=EXP_PATH,
    log_path=EXP_PATH,
    eval_freq=10_000,
    deterministic=True,
    n_eval_episodes=5
)

model = PPO.load(f"{EXP_PATH}/best_model.zip", env=train_env)

model.learn(total_timesteps=TOTAL_ADDITIONAL_STEPS, callback=eval_callback)

model.save(f"{EXP_PATH}/final_model")
train_env.save(f"{EXP_PATH}/vecnormalize.pkl")

train_env.close()
eval_env.close()

print("Final training complete! Yay!")


/Users/ShivenTiwari/Desktop/Bipedal_Walker/venv/lib/python3.13/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=40000, episode_reward=164.44 +/- 135.02
Episode length: 1323.20 +/- 553.60
New best mean reward!
Eval num_timesteps=80000, episode_reward=219.33 +/- 137.13
Episode length: 1344.60 +/- 481.57
New best mean reward!
Eval num_timesteps=120000, episode_reward=288.07 +/- 4.53
Episode length: 1529.80 +/- 58.47
New best mean reward!
Eval num_timesteps=160000, episode_reward=290.05 +/- 2.00
Episode length: 1457.80 +/- 23.87
New best mean reward!
Eval num_timesteps=200000, episode_reward=295.89 +/- 1.92
Episode length: 1398.40 +/- 42.57
New best mean reward!
Eval num_timesteps=240000, episode_reward=272.04 +/- 53.29
Episode length: 1353.60 +/- 80.84
Eval num_timesteps=280000, episode_reward=249.72 +/- 92.97
Episode length: 1231.20 +/- 241.97
Eval num_timesteps=320000, episode_reward=296.98 +/- 2.82
Episode length: 1298.40 +/- 22.95
New best mean reward!
Eval num_timesteps=360000, episode_reward=290.56 +/- 8.83
Episode length: 1386.00 +/- 110.43
Eval num_timesteps=400000, episo

In [ ]:
#Evaluating our model
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy

ENV_ID = "BipedalWalker-v3"
EXP_PATH = "./models_tuning/1_Baseline_LR3e-4_B64"

def make_env():
    return gym.make(ENV_ID)

env = DummyVecEnv([make_env])
env = VecNormalize.load(f"{EXP_PATH}/vecnormalize.pkl", env)
env.training = False
env.norm_reward = False

model = PPO.load(f"{EXP_PATH}/final_model", env=env)

mean_r, std_r = evaluate_policy(model, env, n_eval_episodes=25, deterministic=True) #25 episodes to get reliable results
print(f"Final Eval Reward: {mean_r:.2f} ± {std_r:.2f}")

env.close()

#We notice that our experiment has brought a score of 300.84

Final Eval Reward: 300.84 ± 2.85


In [35]:
#Making GIF for our current walker after base training, reward crafting and 3 experiments
import gymnasium as gym
import imageio
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

ENV_ID = "BipedalWalker-v3"
EXP_PATH = "./models_tuning/1_Baseline_LR3e-4_B64"
OUT_GIF = "./gifs/Final_BipedalWalker.gif"

def make_env_rgb():
    return gym.make(ENV_ID, render_mode="rgb_array")

# VecNormalize env (for correct obs scaling)
vec_env = DummyVecEnv([lambda: gym.make(ENV_ID)])
vec_env = VecNormalize.load(f"{EXP_PATH}/vecnormalize.pkl", vec_env)
vec_env.training = False
vec_env.norm_reward = False

model = PPO.load(f"{EXP_PATH}/final_model", env=vec_env)

# Raw env for frames
raw_env = make_env_rgb()
obs, _ = raw_env.reset(seed=123)

frames = []
for _ in range(1600):
    obs_norm = vec_env.normalize_obs(obs)
    action, _ = model.predict(obs_norm, deterministic=True)
    obs, _, terminated, truncated, _ = raw_env.step(action)
    frames.append(raw_env.render())
    if terminated or truncated:
        break

raw_env.close()
vec_env.close()

imageio.mimsave(OUT_GIF, frames, fps=30)
print("Saved:", OUT_GIF)


Saved: ./gifs/Final_BipedalWalker.gif


In this GIF, we see a significant difference in walking behavior compared to all our previous runs. 

RESULT & SUMMARY (for experiment 3 and others): All groups were trained for 300,000 steps with entropy = 0.0 (experiment 1), observation normalization (experiment 2) enabled, and identical settings. Performance was compared using evaluation on a separate environment.

Among the five groups tested, LR = 3e-4 with batch size = 64 consistently outperformed the others. 

This setup achieved the highest evaluation reward, showed minimal post-peak performance drop, and exhibited smooth improvement over time. More aggressive learning rates led to instability and collapse, while overly cautious configurations learned too slowly within the fixed training budget.
Based on these results, 1_Baseline_LR3e-4_B64 was selected.

The selected model was then extended to more training (about 1 million steps). After continuation, the final policy achieved a mean evaluation reward of 300.84 ± 2.85, indicating near-optimal walking behavior with low variance. Qualitative inspection via recorded GIFs showed a visible improvement in stability and consistency.

In [39]:
#We have already reached a score of 300, but we need additional tests to ensure it consistently gets 300 and above.

import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

ENV_ID = "BipedalWalker-v3"
EXP_PATH = "./models_tuning/1_Baseline_LR3e-4_B64"
MODEL_FILE = f"{EXP_PATH}/final_model.zip"   # or best_model.zip
N_EPISODES = 100

def make_env():
    return gym.make(ENV_ID)

# Load env + VecNormalize
env = DummyVecEnv([make_env])
env = VecNormalize.load(f"{EXP_PATH}/vecnormalize.pkl", env)
env.training = False
env.norm_reward = False

model = PPO.load(MODEL_FILE, env=env)

rewards = []

for _ in range(N_EPISODES):
    obs = env.reset()
    done = False
    ep_r = 0.0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, r, done, info = env.step(action)

        # r is shape (1,), extract scalar
        ep_r += r[0]

    rewards.append(ep_r)

rewards = np.array(rewards, dtype=float)

print(f"mean: {rewards.mean():.2f}")
print(f"std: {rewards.std():.2f}")
print(f"median: {np.median(rewards):.2f}")
print(f"20th percentile: {np.percentile(rewards, 20):.2f}")
print(f"80th percentile: {np.percentile(rewards, 80):.2f}")
print(f"fail rate (<200): {(rewards < 200).mean():.3f}")

env.close()



mean: 270.43
std: 84.32
median: 300.70
20th percentile: 297.47
80th percentile: 302.93
fail rate (<200): 0.130


**Experiment 4 (Reward Normalization Ablation)**

**GOAL**
Reduce catastrophic failures and increase mean by testing whether reward normalization is hurting robustness.

**Concepts involved**
VecNormalize can normalize observations and rewards separately.
Reward normalization changes the scale of the learning signal and can sometimes produce brittle policies (great median, bad tail).
Turning reward norm OFF keeps the agent optimizing the true reward scale, which may reduce rare collapses.

**Variables**
A) Train with norm_reward=True (your current setup)
B) Train with norm_reward=False (new run)
(Keep norm_obs=True in both.)

**Training budget**
300,000 steps (same as your tuning budget). If the tail improves, extend the winner to +500k to +1M.

In [ ]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback

ENV_ID = "BipedalWalker-v3"
BASE_EXP_PATH = "./models_tuning/1_Baseline_LR3e-4_B64"

# New experiment folder
EXP_NAME = "6_RewardNorm_OFF_LR3e-4_B64"
EXP_PATH = os.path.join("./models_tuning", EXP_NAME)
os.makedirs(EXP_PATH, exist_ok=True)

TOTAL_TIMESTEPS = 300_000
EVAL_FREQ = 10_000

# Keep your winner hyperparams fixed
PPO_KWARGS = dict(
    ent_coef=0.0,
    learning_rate=3e-4,
    batch_size=64,
    n_steps=2048,
    verbose=0,
)

def make_env():
    return gym.make(ENV_ID)

if __name__ == "__main__":
    # ---- Train env: obs norm ON, reward norm OFF (this is the ablation) ----
    train_env = DummyVecEnv([make_env for _ in range(4)])
    train_env = VecNormalize(train_env, norm_obs=True, norm_reward=False, clip_obs=10.0)

    # ---- Eval env: obs norm ON, reward norm OFF (true reward) ----
    eval_env = DummyVecEnv([make_env])
    eval_env = VecNormalize(eval_env, norm_obs=True, norm_reward=False, clip_obs=10.0)

    # Share observation normalization stats
    eval_env.obs_rms = train_env.obs_rms

    eval_cb = EvalCallback(
        eval_env,
        best_model_save_path=EXP_PATH,
        log_path=EXP_PATH,
        eval_freq=EVAL_FREQ,
        deterministic=True,
        n_eval_episodes=5
    )

    model = PPO("MlpPolicy", train_env, tensorboard_log="./bipedal_tuning_logs", **PPO_KWARGS)
    model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=eval_cb, tb_log_name=EXP_NAME)

    # Save final + vecnormalize stats (critical for later eval/gifs)
    model.save(os.path.join(EXP_PATH, "final_model.zip"))
    train_env.save(os.path.join(EXP_PATH, "vecnormalize.pkl"))

    train_env.close()
    eval_env.close()

    print("Done:", EXP_NAME)


/Users/ShivenTiwari/Desktop/Bipedal_Walker/venv/lib/python3.13/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=40000, episode_reward=-95.81 +/- 21.14
Episode length: 425.60 +/- 587.87
New best mean reward!
Eval num_timesteps=80000, episode_reward=-42.59 +/- 8.88
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=120000, episode_reward=-81.12 +/- 48.02
Episode length: 707.00 +/- 729.27
Eval num_timesteps=160000, episode_reward=-12.18 +/- 95.81
Episode length: 984.00 +/- 754.47
New best mean reward!
Eval num_timesteps=200000, episode_reward=75.17 +/- 90.38
Episode length: 1328.20 +/- 543.60
New best mean reward!
Eval num_timesteps=240000, episode_reward=42.25 +/- 72.75
Episode length: 1255.40 +/- 424.08
Eval num_timesteps=280000, episode_reward=-42.14 +/- 88.53
Episode length: 495.20 +/- 548.76
Done: 6_RewardNorm_OFF_LR3e-4_B64


In [42]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

ENV_ID = "BipedalWalker-v3"
N_EPISODES = 100

def make_env():
    return gym.make(ENV_ID)

def eval_folder(exp_path: str, model_file: str):
    env = DummyVecEnv([make_env])
    env = VecNormalize.load(f"{exp_path}/vecnormalize.pkl", env)
    env.training = False
    env.norm_reward = False

    model = PPO.load(f"{exp_path}/{model_file}", env=env)

    rewards = []
    for _ in range(N_EPISODES):
        obs = env.reset()
        done = False
        ep_r = 0.0
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, r, done, info = env.step(action)
            ep_r += float(r[0])  # VecEnv reward is shape (1,)
        rewards.append(ep_r)

    rewards = np.array(rewards, dtype=float)
    print(f"\n[{exp_path} | {model_file}]")
    print(f"mean: {rewards.mean():.2f}")
    print(f"std: {rewards.std():.2f}")
    print(f"median: {np.median(rewards):.2f}")
    print(f"20th percentile: {np.percentile(rewards, 20):.2f}")
    print(f"80th percentile: {np.percentile(rewards, 80):.2f}")
    print(f"fail rate (<200): {(rewards < 200).mean():.3f}")

    env.close()

if __name__ == "__main__":
    # Compare your current winner folder vs the ablation folder
    eval_folder("./models_tuning/1_Baseline_LR3e-4_B64", "final_model.zip")      # or best_model.zip
    eval_folder("./models_tuning/6_RewardNorm_OFF_LR3e-4_B64", "final_model.zip") # or best_model.zip



[./models_tuning/1_Baseline_LR3e-4_B64 | final_model.zip]
mean: 282.58
std: 66.19
median: 301.85
20th percentile: 298.52
80th percentile: 304.26
fail rate (<200): 0.080

[./models_tuning/6_RewardNorm_OFF_LR3e-4_B64 | final_model.zip]
mean: 198.42
std: 124.78
median: 284.79
20th percentile: 52.91
80th percentile: 287.32
fail rate (<200): 0.350


RESULT (experiment 4): Training with reward normalization enabled produced substantially more robust policies. While both configurations achieved high median rewards, disabling reward normalization led to a significant increase in variance and catastrophic failures, as reflected by a higher failure rate and a much lower 5th percentile return. These results indicate that reward normalization helps PPO maintain stability and robustness in continuous control tasks such as BipedalWalker.
Hence, I will retain Reward normalization for the final model.

In [ ]:
#EXTENDED TRAINING WITH REWARD NORMALIZATION

import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback

ENV_ID = "BipedalWalker-v3"
EXP_PATH = "./models_tuning/1_Baseline_LR3e-4_B64"   # your best config folder

# Train longer (add steps on top of existing training)
ADDITIONAL_STEPS = 1_000_000
EVAL_FREQ = 10_000

def make_env():
    return gym.make(ENV_ID)

if __name__ == "__main__":
    # -----------------------
    # 1) Train env (VecNormalize stats MUST be loaded)
    # -----------------------
    train_env = DummyVecEnv([make_env for _ in range(4)])
    train_env = VecNormalize.load(os.path.join(EXP_PATH, "vecnormalize.pkl"), train_env)
    train_env.training = True
    train_env.norm_reward = True

    # -----------------------
    # 2) Eval env (true reward)
    # -----------------------
    eval_env = DummyVecEnv([make_env])
    eval_env = VecNormalize.load(os.path.join(EXP_PATH, "vecnormalize.pkl"), eval_env)
    eval_env.training = False
    eval_env.norm_reward = False

    # Keep eval obs normalization synced with training
    eval_env.obs_rms = train_env.obs_rms

    eval_cb = EvalCallback(
        eval_env,
        best_model_save_path=EXP_PATH,
        log_path=EXP_PATH,
        eval_freq=EVAL_FREQ,
        deterministic=True,
        n_eval_episodes=5
    )

    # -----------------------
    # 3) Load best checkpoint and continue training
    # -----------------------
    model = PPO.load(os.path.join(EXP_PATH, "best_model.zip"), env=train_env)

    model.learn(total_timesteps=ADDITIONAL_STEPS, callback=eval_cb)

    # -----------------------
    # 4) Save updated final model + updated normalization stats
    # -----------------------
    model.save(os.path.join(EXP_PATH, "final_model.zip"))
    train_env.save(os.path.join(EXP_PATH, "vecnormalize.pkl"))

    train_env.close()
    eval_env.close()
    print("Extended training complete!")


Eval num_timesteps=40000, episode_reward=306.88 +/- 2.78
Episode length: 1177.00 +/- 43.93
New best mean reward!
Eval num_timesteps=80000, episode_reward=245.64 +/- 123.94
Episode length: 970.60 +/- 294.04
Eval num_timesteps=120000, episode_reward=310.27 +/- 1.26
Episode length: 1121.60 +/- 21.73
New best mean reward!
Eval num_timesteps=160000, episode_reward=230.60 +/- 153.51
Episode length: 876.00 +/- 368.34
Eval num_timesteps=200000, episode_reward=305.61 +/- 1.10
Episode length: 1048.00 +/- 9.78
Eval num_timesteps=240000, episode_reward=303.65 +/- 2.22
Episode length: 1058.80 +/- 29.32
Eval num_timesteps=280000, episode_reward=258.27 +/- 86.92
Episode length: 1008.80 +/- 113.04
Eval num_timesteps=320000, episode_reward=302.11 +/- 1.78
Episode length: 1074.80 +/- 33.87
Eval num_timesteps=360000, episode_reward=240.25 +/- 83.15
Episode length: 958.80 +/- 123.15
Eval num_timesteps=400000, episode_reward=209.89 +/- 125.25
Episode length: 860.80 +/- 237.24
Eval num_timesteps=440000, epi

In [ ]:
#Evaluating our model after reward normalization

import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

ENV_ID = "BipedalWalker-v3"
EXP_PATH = "./models_tuning/1_Baseline_LR3e-4_B64"
MODEL_FILE = "final_model.zip"
N_EPISODES = 200   # using 200 for more stable tail metrics

def make_env():
    return gym.make(ENV_ID)

env = DummyVecEnv([make_env])
env = VecNormalize.load(f"{EXP_PATH}/vecnormalize.pkl", env)
env.training = False
env.norm_reward = False

model = PPO.load(f"{EXP_PATH}/{MODEL_FILE}", env=env)

rewards = []
for _ in range(N_EPISODES):
    obs = env.reset()
    done = False
    ep_r = 0.0
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, r, done, info = env.step(action)
        ep_r += r[0]
    rewards.append(ep_r)

rewards = np.array(rewards, dtype=float)

print("STATS:")
print("-------------------------------------")
print(f"mean: {rewards.mean():.2f}")
print(f"std: {rewards.std():.2f}")
print(f"median: {np.median(rewards):.2f}")
print(f"20th percentile: {np.percentile(rewards, 20):.2f}")
print(f"80th percentile: {np.percentile(rewards, 80):.2f}")
print(f"fail rate (<200): {(rewards < 200).mean():.3f}")
print("-------------------------------------")

env.close()


STATS:
-------------------------------------
mean: 275.82
std: 93.09
median: 309.00
20th percentile: 306.41
80th percentile: 310.83
fail rate (<200): 0.125
-------------------------------------


To improve our model (mainly reduce the fail rate and make it more consistent), we will continue training with 500k more steps and evaluate both best_model.zip and final_model.zip with stats. We shall keep the one with lower fail rate and preferrably higher mean/median. 

In [47]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback

ENV_ID = "BipedalWalker-v3"
EXP_PATH = "./models_tuning/1_Baseline_LR3e-4_B64"

ADDITIONAL_STEPS = 500_000
EVAL_FREQ = 10_000

def make_env():
    return gym.make(ENV_ID)

if __name__ == "__main__":
    # Train env (load VecNormalize stats)
    train_env = DummyVecEnv([make_env for _ in range(4)])
    train_env = VecNormalize.load(os.path.join(EXP_PATH, "vecnormalize.pkl"), train_env)
    train_env.training = True
    train_env.norm_reward = True

    # Eval env (true reward)
    eval_env = DummyVecEnv([make_env])
    eval_env = VecNormalize.load(os.path.join(EXP_PATH, "vecnormalize.pkl"), eval_env)
    eval_env.training = False
    eval_env.norm_reward = False

    # Keep eval obs normalization synced
    eval_env.obs_rms = train_env.obs_rms

    eval_cb = EvalCallback(
        eval_env,
        best_model_save_path=EXP_PATH,
        log_path=EXP_PATH,
        eval_freq=EVAL_FREQ,
        deterministic=True,
        n_eval_episodes=5
    )

    # Continue from current BEST checkpoint (safe choice)
    model = PPO.load(os.path.join(EXP_PATH, "best_model.zip"), env=train_env)

    model.learn(total_timesteps=ADDITIONAL_STEPS, callback=eval_cb)

    # Save updated final + updated normalization stats
    model.save(os.path.join(EXP_PATH, "final_model.zip"))
    train_env.save(os.path.join(EXP_PATH, "vecnormalize.pkl"))

    train_env.close()
    eval_env.close()
    print("Done: +500k steps. Saved final_model.zip and updated vecnormalize.pkl")


/Users/ShivenTiwari/Desktop/Bipedal_Walker/venv/lib/python3.13/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=40000, episode_reward=176.84 +/- 110.77
Episode length: 1400.00 +/- 269.05
New best mean reward!
Eval num_timesteps=80000, episode_reward=300.92 +/- 3.62
Episode length: 1382.00 +/- 50.31
New best mean reward!
Eval num_timesteps=120000, episode_reward=302.47 +/- 2.91
Episode length: 1299.20 +/- 37.87
New best mean reward!
Eval num_timesteps=160000, episode_reward=201.27 +/- 135.29
Episode length: 1036.20 +/- 407.85
Eval num_timesteps=200000, episode_reward=305.89 +/- 1.60
Episode length: 1101.40 +/- 45.83
New best mean reward!
Eval num_timesteps=240000, episode_reward=304.65 +/- 3.12
Episode length: 1116.00 +/- 37.94
Eval num_timesteps=280000, episode_reward=301.75 +/- 3.77
Episode length: 1115.80 +/- 70.43
Eval num_timesteps=320000, episode_reward=240.49 +/- 126.45
Episode length: 946.80 +/- 287.39
Eval num_timesteps=360000, episode_reward=301.25 +/- 7.25
Episode length: 1134.80 +/- 102.33
Eval num_timesteps=400000, episode_reward=273.50 +/- 54.23
Episode length: 10

In [48]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

ENV_ID = "BipedalWalker-v3"
EXP_PATH = "./models_tuning/1_Baseline_LR3e-4_B64"
N_EPISODES = 200  # good for tail stability

def make_env():
    return gym.make(ENV_ID)

def robustness(exp_path: str, model_file: str):
    env = DummyVecEnv([make_env])
    env = VecNormalize.load(f"{exp_path}/vecnormalize.pkl", env)
    env.training = False
    env.norm_reward = False

    model = PPO.load(f"{exp_path}/{model_file}", env=env)

    rewards = []
    for _ in range(N_EPISODES):
        obs = env.reset()
        done = False
        ep_r = 0.0
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, r, done, info = env.step(action)
            ep_r += r[0]
        rewards.append(ep_r)

    rewards = np.array(rewards, dtype=float)

    out = {
        "mean": rewards.mean(),
        "std": rewards.std(),
        "median": np.median(rewards),
        "p05": np.percentile(rewards, 5),
        "fail_rate_<200": (rewards < 200).mean(),
        "min": rewards.min(),
        "max": rewards.max(),
    }

    env.close()
    return out

def print_stats(name, s):
    print(f"\n{name}")
    print(f"mean: {s['mean']:.2f}")
    print(f"std: {s['std']:.2f}")
    print(f"median: {s['median']:.2f}")
    print(f"5th percentile: {s['p05']:.2f}")
    print(f"fail rate (<200): {s['fail_rate_<200']:.3f}")
    print(f"min/max: {s['min']:.2f} / {s['max']:.2f}")

if __name__ == "__main__":
    best = robustness(EXP_PATH, "best_model.zip")
    final = robustness(EXP_PATH, "final_model.zip")

    print_stats("BEST (best_model.zip)", best)
    print_stats("FINAL (final_model.zip)", final)

    # Selection rule: lowest fail rate, then highest mean
    if (final["fail_rate_<200"] < best["fail_rate_<200"]) or (
        final["fail_rate_<200"] == best["fail_rate_<200"] and final["mean"] > best["mean"]
    ):
        print("\nRecommended submission model: final_model.zip")
    else:
        print("\nRecommended submission model: best_model.zip")



BEST (best_model.zip)
mean: 263.05
std: 97.00
median: 304.17
5th percentile: 17.92
fail rate (<200): 0.160
min/max: -68.78 / 310.12

FINAL (final_model.zip)
mean: 209.71
std: 132.28
median: 301.17
5th percentile: -46.12
fail rate (<200): 0.360
min/max: -115.76 / 309.94

Recommended submission model: best_model.zip


EXPERIMENT 5(Clip Range)

**GOAL** 
Test how different PPO clipping values affect training stability and choose a reliable setting for the final model.

**Concepts Involved:** 
PPO limits how much the policy can change at each update. If updates are too large, the walker becomes unstable; if they’re too small, learning slows down. This experiment compares different clip ranges while keeping everything else the same to see which leads to smoother learning.

**Budget**
Training budget: 300k per setting and evaluation every 10k steps



Performance in reinforcement learning can be highly sensitive to random seeds and environmental stochasticity. Evaluating policies across multiple seeds and episodes provides a more reliable estimate of expected performance and reduces the risk of reporting results driven by favorable randomness. (Henederson et al., 2018)

In [49]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback

ENV_ID = "BipedalWalker-v3"
LOG_DIR = "./bipedal_tuning_logs"
BASE_NAME = "EXP5_ClipRange"
TOTAL_TIMESTEPS = 300_000
EVAL_FREQ = 10_000

# keep your winner hyperparams fixed
LR = 3e-4
BATCH = 64
N_STEPS = 2048
ENT = 0.0

CLIP_RUNS = [
    {"name": "A_clip0.2", "clip": 0.2},
    {"name": "B_clip0.1", "clip": 0.1},
    # {"name": "C_clip0.3", "clip": 0.3},  # optional
]

def make_env():
    return gym.make(ENV_ID)

def build_envs():
    train_env = DummyVecEnv([make_env for _ in range(4)])
    train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10.0)

    eval_env = DummyVecEnv([make_env])
    eval_env = VecNormalize(eval_env, norm_obs=True, norm_reward=False, clip_obs=10.0)

    # sync obs normalization so eval matches training scaling
    eval_env.obs_rms = train_env.obs_rms
    return train_env, eval_env

if __name__ == "__main__":
    os.makedirs(LOG_DIR, exist_ok=True)

    for run in CLIP_RUNS:
        name = run["name"]
        clip = run["clip"]

        exp_path = os.path.join("./models_tuning", f"{BASE_NAME}_{name}")
        os.makedirs(exp_path, exist_ok=True)

        train_env, eval_env = build_envs()

        eval_cb = EvalCallback(
            eval_env,
            best_model_save_path=exp_path,
            log_path=exp_path,
            eval_freq=EVAL_FREQ,
            deterministic=True,
            n_eval_episodes=5
        )

        model = PPO(
            "MlpPolicy",
            train_env,
            verbose=0,
            tensorboard_log=LOG_DIR,
            ent_coef=ENT,
            learning_rate=LR,
            batch_size=BATCH,
            n_steps=N_STEPS,
            clip_range=clip
        )

        print(f"Running {BASE_NAME}_{name} with clip_range={clip}")
        model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=eval_cb, tb_log_name=f"{BASE_NAME}_{name}")

        model.save(os.path.join(exp_path, "final_model.zip"))
        train_env.save(os.path.join(exp_path, "vecnormalize.pkl"))

        train_env.close()
        eval_env.close()

    print("Done: Experiment 5 clip range sensitivity.")


Running EXP5_ClipRange_A_clip0.2 with clip_range=0.2
Eval num_timesteps=40000, episode_reward=-95.27 +/- 0.05
Episode length: 64.80 +/- 0.40
New best mean reward!
Eval num_timesteps=80000, episode_reward=-29.03 +/- 11.47
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=120000, episode_reward=38.76 +/- 14.00
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=160000, episode_reward=137.49 +/- 18.61
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=200000, episode_reward=79.77 +/- 89.78
Episode length: 1197.00 +/- 339.24
Eval num_timesteps=240000, episode_reward=242.37 +/- 5.64
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=280000, episode_reward=273.99 +/- 8.74
Episode length: 1582.40 +/- 33.24
New best mean reward!
Running EXP5_ClipRange_B_clip0.1 with clip_range=0.1
Eval num_timesteps=40000, episode_reward=-92.91 +/- 0.30
Episode length: 101.40 +/- 5.00
New best mean reward!
Eval num_time

In [ ]:
#Getting stats for both models over 200 episodes

import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

ENV_ID = "BipedalWalker-v3"
N_EPISODES = 200

EXP_FOLDERS = [
    "./models_tuning/EXP5_ClipRange_A_clip0.2",
    "./models_tuning/EXP5_ClipRange_B_clip0.1",
    # "./models_tuning/EXP5_ClipRange_C_clip0.3",
]

def make_env():
    return gym.make(ENV_ID)

def eval_model(exp_path, model_file):
    env = DummyVecEnv([make_env])
    env = VecNormalize.load(f"{exp_path}/vecnormalize.pkl", env)
    env.training = False
    env.norm_reward = False

    model = PPO.load(f"{exp_path}/{model_file}", env=env)

    rewards = []
    for _ in range(N_EPISODES):
        obs = env.reset()
        done = False
        ep_r = 0.0
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, r, done, info = env.step(action)
            ep_r += r[0]
        rewards.append(ep_r)

    rewards = np.array(rewards, dtype=float)
    env.close()
    return {
        "mean": rewards.mean(),
        "std": rewards.std(),
        "median": np.median(rewards),
        "p80": np.percentile(rewards, 80),
        "p20": np.percentile(rewards, 20),
        "p05": np.percentile(rewards, 5),
        "fail": (rewards < 200).mean(),
        "min": rewards.min(),
        "max": rewards.max(),
    }

def pick_best(exp_path):
    s_best = eval_model(exp_path, "best_model.zip")
    s_final = eval_model(exp_path, "final_model.zip")
    if (s_final["fail"] < s_best["fail"]) or (s_final["fail"] == s_best["fail"] and s_final["mean"] > s_best["mean"]):
        return "final_model.zip", s_final
    return "best_model.zip", s_best

for p in EXP_FOLDERS:
    chosen_name, s = pick_best(p)
    print(f"\n{p} (chosen: {chosen_name})")
    print(f"mean {s['mean']:.2f} | std {s['std']:.2f} | median {s['median']:.2f}")
    print(f"p05 {s['p05']:.2f} | fail<200 {s['fail']:.3f} | min/max {s['min']:.2f}/{s['max']:.2f}")



./models_tuning/EXP5_ClipRange_A_clip0.2 (chosen: best_model.zip)
mean 260.06 | std 54.32 | median 271.78
p05 216.42 | fail<200 0.050 | min/max -80.64/285.82

./models_tuning/EXP5_ClipRange_B_clip0.1 (chosen: best_model.zip)
mean 133.10 | std 13.48 | median 134.33
p05 112.34 | fail<200 1.000 | min/max 90.25/164.09


Result: Experiments showed that overly conservative policy updates (clip range 0.1) severely slowed learning and reduced performance within a fixed training budget. Robustness-based evaluation confirmed that selecting the best checkpoint rather than extending training yielded a final policy with near-optimal median rewards around 300 and improved consistency across episodes.

In [ ]:
#TRAINING OUR MODEL BASED ON ABOVE EXPERIMENTS
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback

ENV_ID = "BipedalWalker-v3"
EXP_PATH = "./models_tuning/FINAL_CLIP0.2_LR3e-4_B64"
os.makedirs(EXP_PATH, exist_ok=True)

LOG_DIR = "./bipedal_tuning_logs"
os.makedirs(LOG_DIR, exist_ok=True)

TOTAL_TIMESTEPS = 1000000
EVAL_FREQ = 10000

# Final chosen hyperparams
LR = 3e-4
BATCH = 64
N_STEPS = 2048
ENT_COEF = 0.0
CLIP_RANGE = 0.2  # explicitly set

def make_env():
    return gym.make(ENV_ID)

if __name__ == "__main__":
    # -----------------------
    # 1) Training env (normalize obs + reward)
    # -----------------------
    train_env = DummyVecEnv([make_env for _ in range(4)])
    train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10.0)
    train_env.training = True
    train_env.norm_reward = True

    # -----------------------
    # 2) Eval env (true reward)
    # -----------------------
    eval_env = DummyVecEnv([make_env])
    eval_env = VecNormalize(eval_env, norm_obs=True, norm_reward=False, clip_obs=10.0)
    eval_env.training = False
    eval_env.norm_reward = False

    # sync obs normalization stats
    eval_env.obs_rms = train_env.obs_rms

    eval_cb = EvalCallback(
        eval_env,
        best_model_save_path=EXP_PATH,
        log_path=EXP_PATH,
        eval_freq=EVAL_FREQ,
        deterministic=True,
        n_eval_episodes=5,
    )

    # -----------------------
    # 3) Train PPO
    # -----------------------
    model = PPO(
        "MlpPolicy",
        train_env,
        verbose=0,
        tensorboard_log=LOG_DIR,
        learning_rate=LR,
        batch_size=BATCH,
        n_steps=N_STEPS,
        ent_coef=ENT_COEF,
        clip_range=CLIP_RANGE,
    )

    model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=eval_cb, tb_log_name="FINAL_CLIP0.2")

    # -----------------------
    # 4) Save model + VecNormalize stats
    # -----------------------
    model.save(os.path.join(EXP_PATH, "final_model.zip"))
    train_env.save(os.path.join(EXP_PATH, "vecnormalize.pkl"))

    train_env.close()
    eval_env.close()

    print("Training complete. Saved final_model.zip and vecnormalize.pkl.")


Eval num_timesteps=40000, episode_reward=-98.96 +/- 1.84
Episode length: 97.80 +/- 5.56
New best mean reward!
Eval num_timesteps=80000, episode_reward=-82.10 +/- 27.28
Episode length: 400.80 +/- 599.60
New best mean reward!
Eval num_timesteps=120000, episode_reward=-25.37 +/- 6.22
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=160000, episode_reward=35.52 +/- 32.38
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=200000, episode_reward=32.09 +/- 94.05
Episode length: 1064.80 +/- 442.04
Eval num_timesteps=240000, episode_reward=43.55 +/- 113.53
Episode length: 930.80 +/- 564.89
New best mean reward!
Eval num_timesteps=280000, episode_reward=187.08 +/- 64.98
Episode length: 1558.40 +/- 83.20
New best mean reward!
Eval num_timesteps=320000, episode_reward=179.58 +/- 124.64
Episode length: 1368.20 +/- 463.60
Eval num_timesteps=360000, episode_reward=53.25 +/- 118.08
Episode length: 913.60 +/- 563.22
Eval num_timesteps=400000, episode_rewa

In [ ]:
#JUST FOR SAFE CASE, CHECKING WHETHER WE NEED THE FINAL OR BEST MODEL IS BETTER


import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

ENV_ID = "BipedalWalker-v3"
EXP_PATH = "./models_tuning/FINAL_CLIP0.2_LR3e-4_B64"
N_EPISODES = 200 #200 runs to get best/reliable results

def make_env():
    return gym.make(ENV_ID)

def eval_model(model_file):
    env = DummyVecEnv([make_env])
    env = VecNormalize.load(f"{EXP_PATH}/vecnormalize.pkl", env)
    env.training = False
    env.norm_reward = False

    model = PPO.load(f"{EXP_PATH}/{model_file}", env=env)

    rewards = []
    for _ in range(N_EPISODES):
        obs = env.reset()
        done = False
        ep_r = 0.0
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, r, done, info = env.step(action)
            ep_r += r[0]
        rewards.append(ep_r)

    rewards = np.array(rewards, dtype=float)
    env.close()
    return {
        "mean": rewards.mean(),
        "std": rewards.std(),
        "median": np.median(rewards),
        "p05": np.percentile(rewards, 5),
        "fail": (rewards < 200).mean(),
        "min": rewards.min(),
        "max": rewards.max(),
    }

def show(name, s):
    print(f"\n{name}")
    print(f"mean: {s['mean']:.2f} | std: {s['std']:.2f} | median: {s['median']:.2f}")
    print(f"p05:  {s['p05']:.2f} | fail<200: {s['fail']:.3f} | min/max: {s['min']:.2f}/{s['max']:.2f}")

best = eval_model("best_model.zip")
final = eval_model("final_model.zip")
show("BEST", best)
show("FINAL", final)

if (final["fail"] < best["fail"]) or (final["fail"] == best["fail"] and final["mean"] > best["mean"]):
    print("\nUse: final_model.zip")
else:
    print("\nUse: best_model.zip")



BEST
mean: 294.61 | std: 41.13 | median: 301.35
p05:  295.65 | fail<200: 0.030 | min/max: -37.60/308.18

FINAL
mean: 262.41 | std: 97.15 | median: 310.82
p05:  37.84 | fail<200: 0.220 | min/max: -45.60/318.80

Use: best_model.zip


In [ ]:
#SAVING THE GIF OF OUR MODEL
import os
import gymnasium as gym
import imageio
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# CONFIG
ENV_ID = "BipedalWalker-v3"
EXP_PATH = "./models_tuning/FINAL_CLIP0.2_LR3e-4_B64"   # folder with vecnormalize.pkl + best_model.zip
MODEL_FILE = "best_model.zip"

OUT_DIR = "./gifs"
OUT_GIF = os.path.join(OUT_DIR, "bipedal_FINAL_best.gif")

SEED = 123
MAX_STEPS = 1600
FPS = 12

os.makedirs(OUT_DIR, exist_ok=True)

def make_env_rgb():
    return gym.make(ENV_ID, render_mode="rgb_array")

# VecNormalize must be applied exactly like training
env = DummyVecEnv([make_env_rgb])
env = VecNormalize.load(os.path.join(EXP_PATH, "vecnormalize.pkl"), env)
env.training = False
env.norm_reward = False

model = PPO.load(os.path.join(EXP_PATH, MODEL_FILE), env=env)

frames = []
obs = env.reset()
done = False
steps = 0

while (not done) and (steps < MAX_STEPS):
    frame = env.render()  # for rgb_array this returns an image
    if frame is not None:
        frames.append(frame)

    action, _ = model.predict(obs, deterministic=True)
    obs, r, done, info = env.step(action)
    steps += 1

# add a couple extra frames at the end
for _ in range(10):
    frame = env.render()
    if frame is not None:
        frames.append(frame)

imageio.mimsave(OUT_GIF, frames, fps=FPS)
env.close()

print(f"Saved GIF: {OUT_GIF} ({len(frames)} frames)")


Saved GIF: ./gifs/bipedal_FINAL_best.gif (1111 frames)


Our gif has good stats currently but its GIF represents a weird walking behavior. Now we will try to repair that

In [ ]:
#Training our model again with 1M steps without changing anything
import os
import gymnasium as gym
import numpy as np

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecMonitor
from stable_baselines3.common.callbacks import EvalCallback

ENV_ID = "BipedalWalker-v3"

# Output
EXP_PATH = "./models_tuning/FINAL_SHAPED_GAIT"
LOG_DIR = "./bipedal_tuning_logs"
os.makedirs(EXP_PATH, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# Training
TOTAL_TIMESTEPS = 1_000_000
EVAL_FREQ = 10_000

# PPO (your chosen defaults)
LR = 3e-4
BATCH = 64
N_STEPS = 2048
ENT_COEF = 0.0
CLIP_RANGE = 0.2


class GaitShaping(gym.Wrapper):
    """
    Adds small shaping for nicer gait (faster, upright, smoother).
    We DO NOT change termination logic; only add to reward.

    Tune weights gently. Too large can destabilize.
    """
    def __init__(
        self,
        env,
        vx_k=0.30,          # reward forward velocity
        lean_k=0.10,        # penalize hull angle magnitude
        effort_k=0.0008,    # penalize big action magnitudes (smoothness)
        jitter_k=0.02       # penalize fast action changes (less jerky)
    ):
        super().__init__(env)
        self.vx_k = vx_k
        self.lean_k = lean_k
        self.effort_k = effort_k
        self.jitter_k = jitter_k
        self.prev_action = None

    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        self.prev_action = None
        return obs, info

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)

        # Obs structure in BipedalWalker-v3:
        # obs[2] is forward velocity, obs[0] is hull angle (typical for this env)
        # (Gymnasium versions match this convention)
        hull_angle = float(obs[0])
        vx = float(obs[2])

        # Smoothness/effort penalties
        effort = float(np.sum(np.square(action)))
        if self.prev_action is None:
            jitter = 0.0
        else:
            jitter = float(np.sum(np.square(action - self.prev_action)))
        self.prev_action = action.copy()

        shaped = reward
        shaped += self.vx_k * vx
        shaped -= self.lean_k * abs(hull_angle)
        shaped -= self.effort_k * effort
        shaped -= self.jitter_k * jitter

        return obs, shaped, terminated, truncated, info


def make_train_env():
    env = gym.make(ENV_ID)
    env = GaitShaping(env)          # <-- shaping only during training
    return env


def make_eval_env():
    env = gym.make(ENV_ID)          # <-- NO shaping during eval (true reward)
    return env


if __name__ == "__main__":
    # Vectorized train env
    train_env = DummyVecEnv([make_train_env for _ in range(4)])
    train_env = VecMonitor(train_env)
    train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10.0)

    # Vectorized eval env (true reward)
    eval_env = DummyVecEnv([make_eval_env])
    eval_env = VecMonitor(eval_env)
    eval_env = VecNormalize(eval_env, norm_obs=True, norm_reward=False, clip_obs=10.0)

    # Share obs normalization stats so eval uses the same scaling
    eval_env.obs_rms = train_env.obs_rms

    eval_cb = EvalCallback(
        eval_env,
        best_model_save_path=EXP_PATH,
        log_path=EXP_PATH,
        eval_freq=EVAL_FREQ,
        deterministic=True,
        n_eval_episodes=5
    )

    model = PPO(
        "MlpPolicy",
        train_env,
        verbose=0,
        tensorboard_log=LOG_DIR,
        learning_rate=LR,
        batch_size=BATCH,
        n_steps=N_STEPS,
        ent_coef=ENT_COEF,
        clip_range=CLIP_RANGE,
    )

    model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=eval_cb, tb_log_name="FINAL_SHAPED_GAIT")

    # Save
    model.save(os.path.join(EXP_PATH, "final_model.zip"))
    train_env.save(os.path.join(EXP_PATH, "vecnormalize.pkl"))

    train_env.close()
    eval_env.close()
    print("Done. Saved best_model.zip (via EvalCallback), final_model.zip, and vecnormalize.pkl")


/Users/ShivenTiwari/Desktop/Bipedal_Walker/venv/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Eval num_timesteps=40000, episode_reward=-31.23 +/- 1.56
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=80000, episode_reward=-50.75 +/- 1.33
Episode length: 1600.00 +/- 0.00
Eval num_timesteps=120000, episode_reward=-5.18 +/- 22.23
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=160000, episode_reward=74.95 +/- 39.90
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=200000, episode_reward=126.41 +/- 7.83
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=240000, episode_reward=150.32 +/- 2.65
Episode length: 1600.00 +/- 0.00
New best mean reward!
Eval num_timesteps=280000, episode_reward=66.91 +/- 110.97
Episode length: 1396.80 +/- 406.40
Eval num_timesteps=320000, episode_reward=-93.78 +/- 3.70
Episode length: 105.40 +/- 16.93
Eval num_timesteps=360000, episode_reward=-42.32 +/- 50.67
Episode length: 524.00 +/- 411.79
Eval num_timesteps=400000, episode_reward=143.20 +/- 110.15
Episode 

In [ ]:
#Comparing best and final once again

import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

ENV_ID = "BipedalWalker-v3"
EXP_PATH = "./models_tuning/FINAL_SHAPED_GAIT"
N_EPISODES = 200

def make_env():
    return gym.make(ENV_ID)

def eval_file(model_file: str):
    env = DummyVecEnv([make_env])
    env = VecNormalize.load(f"{EXP_PATH}/vecnormalize.pkl", env)
    env.training = False
    env.norm_reward = False

    model = PPO.load(f"{EXP_PATH}/{model_file}", env=env)

    rewards = []
    for _ in range(N_EPISODES):
        obs = env.reset()
        done = False
        ep_r = 0.0
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, r, done, info = env.step(action)
            ep_r += r[0]
        rewards.append(ep_r)

    rewards = np.array(rewards, dtype=float)
    env.close()
    return {
        "mean": rewards.mean(),
        "std": rewards.std(),
        "median": np.median(rewards),
        "p05": np.percentile(rewards, 5),
        "fail": (rewards < 200).mean(),
        "min": rewards.min(),
        "max": rewards.max(),
    }

def show(name, s):
    print(f"\n{name}")
    print(f"mean: {s['mean']:.2f} | std: {s['std']:.2f} | median: {s['median']:.2f}")
    print(f"p05:  {s['p05']:.2f} | fail<200: {s['fail']:.3f} | min/max: {s['min']:.2f}/{s['max']:.2f}")

best = eval_file("best_model.zip")
final = eval_file("final_model.zip")
show("BEST", best)
show("FINAL", final)



BEST
mean: 284.83 | std: 80.72 | median: 309.37
p05:  37.83 | fail<200: 0.090 | min/max: -71.43/313.60

FINAL
mean: 285.64 | std: 68.29 | median: 306.87
p05:  80.78 | fail<200: 0.095 | min/max: -55.83/312.08


In [ ]:
#Continue to train model and look at each checkpoint(every 50k steps) to find best model
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback

ENV_ID = "BipedalWalker-v3"

# Use your best proven setup folder (has vecnormalize.pkl + best_model.zip)
BASE_EXP_PATH = "./models_tuning/FINAL_CLIP0.2_LR3e-4_B64"

# Where to save checkpoint-hunt run
HUNT_PATH = "./models_tuning/HUNT_walk_quality"
os.makedirs(HUNT_PATH, exist_ok=True)

LOG_DIR = "./bipedal_tuning_logs"
os.makedirs(LOG_DIR, exist_ok=True)

ADDITIONAL_STEPS = 500_000
EVAL_FREQ = 10_000
CKPT_FREQ = 50_000  # save checkpoint every 50k steps

def make_env():
    return gym.make(ENV_ID)

if __name__ == "__main__":
    # Train env (load existing normalization)
    train_env = DummyVecEnv([make_env for _ in range(4)])
    train_env = VecNormalize.load(os.path.join(BASE_EXP_PATH, "vecnormalize.pkl"), train_env)
    train_env.training = True
    train_env.norm_reward = True

    # Eval env (true reward)
    eval_env = DummyVecEnv([make_env])
    eval_env = VecNormalize.load(os.path.join(BASE_EXP_PATH, "vecnormalize.pkl"), eval_env)
    eval_env.training = False
    eval_env.norm_reward = False
    eval_env.obs_rms = train_env.obs_rms

    eval_cb = EvalCallback(
        eval_env,
        best_model_save_path=HUNT_PATH,
        log_path=HUNT_PATH,
        eval_freq=EVAL_FREQ,
        deterministic=True,
        n_eval_episodes=5
    )

    ckpt_cb = CheckpointCallback(
        save_freq=CKPT_FREQ,
        save_path=HUNT_PATH,
        name_prefix="ckpt"
    )

    # Continue from best checkpoint you already have
    model = PPO.load(os.path.join(BASE_EXP_PATH, "best_model.zip"), env=train_env)

    model.learn(
        total_timesteps=ADDITIONAL_STEPS,
        callback=[eval_cb, ckpt_cb],
        tb_log_name="HUNT_walk_quality"
    )

    # Save final + updated VecNormalize stats for this hunt run
    model.save(os.path.join(HUNT_PATH, "final_model.zip"))
    train_env.save(os.path.join(HUNT_PATH, "vecnormalize.pkl"))

    train_env.close()
    eval_env.close()
    print("Done. Saved checkpoints ckpt_*.zip, best_model.zip, final_model.zip, vecnormalize.pkl in:", HUNT_PATH)


/Users/ShivenTiwari/Desktop/Bipedal_Walker/venv/lib/python3.13/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=40000, episode_reward=237.83 +/- 128.92
Episode length: 954.00 +/- 309.48
New best mean reward!
Eval num_timesteps=80000, episode_reward=197.03 +/- 148.01
Episode length: 781.00 +/- 305.57
Eval num_timesteps=120000, episode_reward=220.61 +/- 112.94
Episode length: 787.80 +/- 179.98
Eval num_timesteps=160000, episode_reward=204.78 +/- 140.84
Episode length: 769.20 +/- 239.69
Eval num_timesteps=200000, episode_reward=-26.48 +/- 166.29
Episode length: 274.40 +/- 353.32
Eval num_timesteps=240000, episode_reward=308.49 +/- 1.52
Episode length: 912.80 +/- 11.34
New best mean reward!
Eval num_timesteps=280000, episode_reward=314.66 +/- 1.42
Episode length: 824.20 +/- 11.75
New best mean reward!
Eval num_timesteps=320000, episode_reward=289.03 +/- 44.05
Episode length: 843.20 +/- 24.54
Eval num_timesteps=360000, episode_reward=267.13 +/- 90.45
Episode length: 745.80 +/- 109.80
Eval num_timesteps=400000, episode_reward=289.00 +/- 49.46
Episode length: 796.80 +/- 29.58
Eval nu

In [ ]:
#RANKS ALL THE SAVED MODELS
import os
import glob
import json
import numpy as np
import pandas as pd
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy

ENV_ID = "BipedalWalker-v3"

HUNT_PATH = "./models_tuning/HUNT_walk_quality"
N_EVAL_EPISODES = 10
SEED = 123
DETERMINISTIC = True

def make_env(seed=None):
    def _thunk():
        env = gym.make(ENV_ID)
        if seed is not None:
            env.reset(seed=seed)
        return env
    return _thunk

def load_eval_env(hunt_path: str):
    """
    Creates an eval env with the *hunt run's* vecnormalize.pkl
    and ensures we evaluate on true reward.
    """
    venv = DummyVecEnv([make_env(seed=SEED)])
    venv = VecNormalize.load(os.path.join(hunt_path, "vecnormalize.pkl"), venv)
    venv.training = False
    venv.norm_reward = False
    return venv

def list_candidate_models(hunt_path: str):
    candidates = []
    for name in ["best_model.zip", "final_model.zip"]:
        p = os.path.join(hunt_path, name)
        if os.path.exists(p):
            candidates.append(p)
    candidates += sorted(glob.glob(os.path.join(hunt_path, "ckpt_*.zip")))
    # dedupe while preserving order
    seen = set()
    out = []
    for p in candidates:
        if p not in seen:
            out.append(p)
            seen.add(p)
    return out

def evaluate_one(model_path: str, eval_env):
    model = PPO.load(model_path, env=eval_env)

    # evaluate_policy returns (mean_reward, std_reward) by default
    mean_r, std_r = evaluate_policy(
        model,
        eval_env,
        n_eval_episodes=N_EVAL_EPISODES,
        deterministic=DETERMINISTIC,
        return_episode_rewards=False,
    )

    # Also get per-episode rewards/lengths for more detailed stats
    ep_rewards, ep_lengths = evaluate_policy(
        model,
        eval_env,
        n_eval_episodes=N_EVAL_EPISODES,
        deterministic=DETERMINISTIC,
        return_episode_rewards=True,
    )

    ep_rewards = np.array(ep_rewards, dtype=float)
    ep_lengths = np.array(ep_lengths, dtype=float)

    stats = {
        "model_path": model_path,
        "mean_return": float(mean_r),
        "std_return": float(std_r),
        "min_return": float(ep_rewards.min()),
        "p25_return": float(np.percentile(ep_rewards, 25)),
        "median_return": float(np.median(ep_rewards)),
        "p75_return": float(np.percentile(ep_rewards, 75)),
        "max_return": float(ep_rewards.max()),
        "mean_ep_len": float(ep_lengths.mean()),
        "std_ep_len": float(ep_lengths.std(ddof=0)),
    }
    return stats

def main():
    assert os.path.isdir(HUNT_PATH), f"Not found: {HUNT_PATH}"
    vec_path = os.path.join(HUNT_PATH, "vecnormalize.pkl")
    assert os.path.exists(vec_path), f"Missing VecNormalize stats: {vec_path}"

    model_paths = list_candidate_models(HUNT_PATH)
    assert len(model_paths) > 0, f"No models found in {HUNT_PATH}"

    eval_env = load_eval_env(HUNT_PATH)

    rows = []
    for mp in model_paths:
        s = evaluate_one(mp, eval_env)
        rows.append(s)
        print(f"Evaluated: {os.path.basename(mp)} | mean_return={s['mean_return']:.2f} std={s['std_return']:.2f}")

    eval_env.close()

    df = pd.DataFrame(rows).sort_values(["mean_return", "median_return"], ascending=False).reset_index(drop=True)

    out_csv = os.path.join(HUNT_PATH, "model_eval_ranking.csv")
    out_json = os.path.join(HUNT_PATH, "model_eval_ranking.json")
    df.to_csv(out_csv, index=False)
    with open(out_json, "w") as f:
        json.dump(df.to_dict(orient="records"), f, indent=2)

    best = df.iloc[0].to_dict()
    print("\n=== BEST MODEL (by mean_return) ===")
    print(f"Path: {best['model_path']}")
    print(f"Mean return: {best['mean_return']:.2f}  |  Std: {best['std_return']:.2f}")
    print(f"Median: {best['median_return']:.2f} | IQR: [{best['p25_return']:.2f}, {best['p75_return']:.2f}]")
    print(f"Mean ep len: {best['mean_ep_len']:.1f}")

    print(f"\nSaved rankings:\n- {out_csv}\n- {out_json}")

if __name__ == "__main__":
    main()


Evaluated: best_model.zip | mean_return=112.49 std=118.86
Evaluated: final_model.zip | mean_return=311.87 std=1.27
Evaluated: ckpt_200000_steps.zip | mean_return=294.84 std=39.46
Evaluated: ckpt_400000_steps.zip | mean_return=211.87 std=128.04

=== BEST MODEL (by mean_return) ===
Path: ./models_tuning/HUNT_walk_quality/final_model.zip
Mean return: 311.87  |  Std: 1.27
Median: 311.77 | IQR: [310.86, 313.25]
Mean ep len: 778.8

Saved rankings:
- ./models_tuning/HUNT_walk_quality/model_eval_ranking.csv
- ./models_tuning/HUNT_walk_quality/model_eval_ranking.json


In [ ]:
#Generate GIF and video of final model

import os
import pandas as pd
import gymnasium as gym
import imageio

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor

ENV_ID = "BipedalWalker-v3"
HUNT_PATH = "./models_tuning/HUNT_walk_quality"

RANKING_CSV = os.path.join(HUNT_PATH, "model_reeval_ranking.csv")
VECNORM_PATH = os.path.join(HUNT_PATH, "vecnormalize.pkl")

OUT_DIR = os.path.join(HUNT_PATH, "renders")
os.makedirs(OUT_DIR, exist_ok=True)

SEED = 123
MAX_STEPS = 1600
MP4_FPS = 30
GIF_FPS = 12

def make_rgb_env():
    env = gym.make(ENV_ID, render_mode="rgb_array")
    env = Monitor(env)
    return env

def load_eval_env_rgb(vecnorm_path: str, seed: int):
    venv = DummyVecEnv([lambda: make_rgb_env()])
    venv = VecNormalize.load(vecnorm_path, venv)
    venv.training = False
    venv.norm_reward = False
    venv.seed(seed)  # seed the VecEnv (more reliable than seeding inside thunk)
    return venv

def main():
    assert os.path.exists(RANKING_CSV), f"Missing ranking CSV: {RANKING_CSV}"
    assert os.path.exists(VECNORM_PATH), f"Missing vecnormalize.pkl: {VECNORM_PATH}"

    df = pd.read_csv(RANKING_CSV)
    assert len(df) > 0, "Ranking CSV is empty."

    best_model_path = str(df.loc[0, "model_path"])
    assert os.path.exists(best_model_path), f"Best model not found: {best_model_path}"

    print("Selected best model:", best_model_path)

    env = load_eval_env_rgb(VECNORM_PATH, SEED)
    model = PPO.load(best_model_path, env=env, device="cpu")

    frames = []
    obs = env.reset()
    ep_return = 0.0
    steps = 0

    for _ in range(MAX_STEPS):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)  # <-- 4 outputs

        ep_return += float(reward[0])
        steps += 1

        frames.append(env.venv.envs[0].render())

        if bool(done[0]):
            break

    env.close()

    base = os.path.splitext(os.path.basename(best_model_path))[0]
    mp4_path = os.path.join(OUT_DIR, f"{base}_seed{SEED}.mp4")
    gif_path = os.path.join(OUT_DIR, f"{base}_seed{SEED}.gif")

    imageio.mimsave(mp4_path, frames, fps=MP4_FPS)
    imageio.mimsave(gif_path, frames, fps=GIF_FPS)

    print(f"Episode steps: {steps}")
    print(f"Episode return: {ep_return:.2f}")
    print("Saved:")
    print(" -", mp4_path)
    print(" -", gif_path)

if __name__ == "__main__":
    main()


Selected best model: ./models_tuning/HUNT_walk_quality/final_model.zip


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Episode steps: 773
Episode return: 312.83
Saved:
 - ./models_tuning/HUNT_walk_quality/renders/final_model_seed123.mp4
 - ./models_tuning/HUNT_walk_quality/renders/final_model_seed123.gif


In [2]:
#Display stats of our final model

import os
import numpy as np
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

ENV_ID = "BipedalWalker-v3"
HUNT_PATH = "./models_tuning/HUNT_walk_quality"

MODEL_PATH = os.path.join(HUNT_PATH, "final_model.zip")
VECNORM_PATH = os.path.join(HUNT_PATH, "vecnormalize.pkl")

N_EVAL_EPISODES = 100
DETERMINISTIC = True
SEED = 123

# --- checks ---
assert os.path.exists(MODEL_PATH), "final_model.zip not found"
assert os.path.exists(VECNORM_PATH), "vecnormalize.pkl not found"

def make_env():
    env = gym.make(ENV_ID)
    env = Monitor(env)
    return env

# --- env ---
env = DummyVecEnv([lambda: make_env()])
env = VecNormalize.load(VECNORM_PATH, env)
env.training = False
env.norm_reward = False
env.seed(SEED)

# --- model ---
model = PPO.load(MODEL_PATH, env=env, device="cpu")

# --- evaluate ---
ep_rewards, _ = evaluate_policy(
    model,
    env,
    n_eval_episodes=N_EVAL_EPISODES,
    deterministic=DETERMINISTIC,
    return_episode_rewards=True,
)

env.close()

ep_rewards = np.array(ep_rewards, dtype=float)

# --- stats ---
mean = ep_rewards.mean()
std = ep_rewards.std(ddof=0)
median = np.median(ep_rewards)
p20 = np.percentile(ep_rewards, 20)
p80 = np.percentile(ep_rewards, 80)
fail_rate = np.mean(ep_rewards < 200)

# --- display ---
print("\nSTATS:")
print("-------------------------------------")
print(f"mean: {mean:.2f}")
print(f"std: {std:.2f}")
print(f"median: {median:.2f}")
print(f"20th percentile: {p20:.2f}")
print(f"80th percentile: {p80:.2f}")
print(f"fail rate (<200): {fail_rate:.3f}")
print("-------------------------------------")


STATS:
-------------------------------------
mean: 300.99
std: 46.86
median: 311.73
20th percentile: 310.46
80th percentile: 313.14
fail rate (<200): 0.060
-------------------------------------


Yay!! we have formally passed the BipedalWalker challenge by getting a average score of 300 on 100 consecutive runs